<a href="https://colab.research.google.com/github/BakhturinaPolina/goodreads-romance-research/blob/main/scraping_ratings_information_expanded_romantic_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required packages if you haven't already
# BeautifulSoup is for web scraping, and requests is for making HTTP requests.
!pip install beautifulsoup4
!pip install requests
!pip install lxml
!pip install selenium
# Install geckodriver-autoinstaller
!pip install geckodriver-autoinstaller
# Install webdriver_manager
!pip install webdriver_manager
# Install Pandas
!pip install pandas
# Install regex
!pip install regex
# Install credentials (Note: 'credentials' is not a standard Python package, so this is a placeholder. Replace it with the actual package name if applicable.)
!pip install credentials

In [ ]:
# Import the required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
from requests.exceptions import RequestException
import time
import argparse
from datetime import datetime
import json
import os
import re
from urllib.request import urlopen
from urllib.error import HTTPError
import os.path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

In [ ]:
# Define a list of books and their authors that you're interested in
books = [
    {'author': 'Lauren Asher', 'titles': ['Terms and Conditions', 'The Fine Print']},
    {'author': 'Sarina Bowen', 'titles': ['Brooklynaire', 'Hard Hitter', 'Pipe Dreams']},
    {'author': 'Jessica Clare', 'titles': ['Stranded with a Billionaire', 'Beauty and the Billionaire', 'The Wrong Billionaire’s Bed']},
    {'author': 'Sylvia Day', 'titles': ['Bared to you', 'Reflected in You', 'Entwined with You', 'Captivated by You', 'One with You']},
    {'author': 'J.T. Geissinger', 'titles': ['Burn for You', 'Melt for you', 'Ache for you']},
    {'author': 'Pipa Grant', 'titles': ['The Last Eligible Billionaire', 'Mister McHottie', 'Crazy for Loving You']},
    {'author': 'Melanie Harlow', 'titles': ['Tease', 'Irresistible', 'Call Me Crazy']},
    {'author': 'E.L. James', 'titles': ['Fifty Shades of Grey', 'Fifty Shades Darker', 'Fifty Shades Freed']},
    {'author': 'J. Kenner', 'titles': ['Release Me', 'Claim Me', 'Complete Me']},
    {'author': 'Lisa Kleypas', 'titles': ['Sugar Daddy', 'Blue-Eyed Devil', 'Smooth Talking Stranger']},
    {'author': 'Jessica Lemmon', 'titles': ['The Bastard Billionaire', 'The Billionaire Next Door', 'The Billionaire Bachelor']},
    {'author': 'K.A. Linde', 'titles': ['The Wright Brother', 'The Wright Boss', 'The Wright Mistake']},
    {'author': 'Tia Louise', 'titles': ['Trouble', 'Boss of Me', 'Make Me Yours']},
    {'author': 'J.E. Malpas', 'titles': ['This Man', 'Beneath This Man', 'This Man Confessed', 'With This Man']},
    {'author': 'Meghan March', 'titles': ['Dirty Billionaire', 'Dirty Pleasures', 'Dirty Together']},
    {'author': 'Annika Martin', 'titles': ['Most Eligible Billionaire', 'The Billionaire’s Wake-up-call Girl', 'Breaking the Billionaire’s Rules']},
    {'author': 'Max Monroe', 'titles': ['Tapping the Billionaire', 'Banking the Billionaire', 'Scoring the Billionaire']},
    {'author': 'Laurelin Paige', 'titles': ['Fixed on You', 'Found in You', 'Forever with You']},
    {'author': 'Stella Rhys', 'titles': ['Bad Boss', 'Sweet Spot', 'Dirty Deeds']},
    {'author': 'S.L. Scott', 'titles': ['The Billionaire’s Obsession: Simon', 'The Billionaire’s Obsession: Sam', 'The Billionaire’s Salvation: Max']},
    {'author': 'L.J. Shen', 'titles': ['The Villain', 'The Hunter', 'The Monster']},
    {'author': 'T.L. Swan', 'titles': ['The Take Over', 'The Stopover', 'The Casanova']}
]

print(len(books))

In [ ]:
[
    {"author": "Sarina Bowen", "titles": ["Brooklynaire", "Hard Hitter", "Pipe Dreams"]},
    {"author": "J. Clare", "titles": ["Stranded with a Billionaire", "Beauty and the Billionaire", "The Wrong Billionaire’s Bed"]},
    {"author": "J.T. Geissinger", "titles": ["Burn for You", "Melt for You", "Ache for You"]},
    {"author": "Pipa Grant", "titles": ["The Last Eligible Billionaire", "Mister McHottie", "Crazy for Loving You: A Bluewater Billionaires Romantic Comedy"]},
    {"author": "Melanie Harlow", "titles": ["Tease", "Irresistible", "Call Me Crazy"]},
    {"author": "Lisa Kleypas", "titles": ["Sugar Daddy", "Blue-Eyed Devil", "Smooth Talking Stranger"]},
    {"author": "Jessica Lemmon", "titles": ["The Bastard Billionaire", "The Billionaire Next Door", "The Billionaire Bachelor"]},
    {"author": "K.A. Linde", "titles": ["The Wright Brother", "The Wright Boss", "The Wright Mistake"]},
    {"author": "Tia Louise", "titles": ["Trouble", "Boss of Me", "Make Me Yours"]},
    {"author": "Annika Martin", "titles": ["Most Eligible Billionaire", "The Billionaire's Wake-up-call Girl", "Breaking the Billionaire's Rules"]},
    {"author": "Max Monroe", "titles": ["Tapping the Billionaire", "Banking the Billionaire", "Scoring the Billionaire"]},
    {"author": "Stella Rhys", "titles": ["Bad Boss", "Sweet Spot", "Dirty Deeds"]},
    {"author": "S.L. Scott", "titles": ["The Billionaire's Obsession: Simon", "The Billionaire’s Obsession: Sam", "The Billionaire's Salvation: Max"]},
    {"author": "L.J. Shen", "titles": ["The Villain", "The Hunter", "The Monster"]},
    {"author": "T.L. Swan", "titles": ["The Take-Over", "The Stopover", "The Casanova"]},
    {"author": "Ana Huang", "titles": ["King of Wrath", "Twisted Love", "Twisted Lies"]},
    {"author": "Meghan Quinn", "titles": ["A Not So Meet Cute", "So Not Meant To Be", "A Long Time Coming"]},
    {"author": "Catharina Maura", "titles": ["The Wrong Bride", "The Temporary Wife", "The Unwanted Marriage"]},
    {"author": "Sara Cate", "titles": ["Praise", "Eyes on Me", "Highest Bidder"]},
    {"author": "Shain Rose", "titles": ["Between Commitment and Betrayal", "Between Love and Loathing", "Reverie"]},
    {"author": "Jennifer Probst", "titles": ["The Marriage Bargain", "The Marriage Arrangement", "The Marriage Trap"]},
    {"author": "Laurelin Paige", "titles": ["Brutal Secret", "Brutal Billionaire", "Dirty Filthy Billionaire"]},
    {"author": "Leslie North", "titles": ["The Billionaire's Ex-Wife", "The Billionaire’s Pregnant Fling", "The Billionaire's Sexy Rival"]},
    {"author": "Diane Alberts", "titles": ["Beauty and the Boss", "The Prince's Bride", "An Accidental Date with a Billionaire"]},
    {"author": "Melody Anne", "titles": ["The Tycoon's Revenge", "The Tycoon's Vacation", "The Tycoon's Proposal"]},
    {"author": "L. Steele", "titles": ["The Billionaire's Fake Wife", "The Billionaire's Secret", "The Billionaire's Christmas Bride"]},
    {"author": "Kendra Little", "titles": ["Bedding The Billionaire", "Billionaire Bad Boy", "Suddenly Sexy"]},
    {"author": "Sam Crescent", "titles": ["Billionaire Protector", "Billionaire Beast", "Billionaire Hero"]},
    {"author": "Sandi Lynn", "titles": ["Carter Grayson: A Billionaire Romance", "Chase Calloway", "Jamieson Finn"]},
    {"author": "S. Ann Cole", "titles": ["Mr. Mysterious In Black", "Love Has A Name", "Choose You"]},
    {"author": "Louise Bay", "titles": ["King of Wall Street", "Park Avenue Prince", "Duke of Manhattan"]},
    {"author": "Katy Evans", "titles": ["Tycoon", "Mogul", "Boss"]},
    {"author": "Emma Bray", "titles": ["Her Billionaire Stalker", "Stalked by the Billionaire", "Unmasking the Billionaire"]},
    {"author": "Alexa Riley", "titles": ["Everything for Her", "His Alone", "Devotion"]},
    {"author": "Penny Wylder", "titles": ["The Billionaire's Secret Babies", "The Billionaire's Virgin", "The Billionaire’s Toy"]}
]

In [ ]:
# Initialize an empty list to store the data
book_details = []

# User agent string to mimic a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Loop through each author and their books to find the links
for book in books:
    author = book['author']

    for title in book['titles']:
        # Replace spaces with '+' to form the URL
        search_title = title.replace(' ', '+')

        # Generate the Goodreads search URL
        search_url = f"https://www.goodreads.com/search?utf8=%E2%9C%93&q={search_title}&search_type=books"

        # Fetch the raw HTML content of the search page
        search_page = requests.get(search_url, headers=headers)

        if search_page.status_code == 200:
            search_soup = BeautifulSoup(search_page.content, 'html.parser')

            # Attempt to find the book's link
            try:
                book_url_partial = search_soup.find('a', {'class': 'bookTitle'})['href']
                book_url_clean = "https://www.goodreads.com" + book_url_partial.split("?")[0]  # The URL is cleaned here

                # Extract book ID from the URL
                book_id = re.search(r'(\d+)', book_url_partial).group(1)

                book_details.append({
                    'ID': book_id,
                    'Author': author,
                    'Title': title,
                    'URL': book_url_clean
                })
            except TypeError:
                print(f"Could not find the book: {title} by {author}")
        else:
            print(f"Failed to retrieve the search page for {title} by {author}")

        # Wait for a few seconds before the next request
        time.sleep(2)

# Print the list of scraped book details
print(len(book_details))
print(book_details)

In [ ]:
import os.path
from selenium import webdriver

# Check if chromedriver is already installed
if not os.path.isfile('/usr/bin/chromedriver'):
    # Install Selenium
    !pip install selenium
    # Set up ChromeDriver
    !apt-get update
    !apt install -y chromium-chromedriver

# Check if it's necessary to copy chromedriver (it may already be in /usr/bin)
if not os.path.exists('/usr/bin/chromedriver'):
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize WebDriver
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.goodreads.com/')
print(driver.title)  # Should print "Google"

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

def extract_characters(driver):
    try:
        # Locate the button
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[@aria-label='Show all items in the list']"))
        )
    except TimeoutException:
        print("Failed to locate the button.")
        print(driver.page_source)  # Print the current HTML content
        return None

    # Scroll the button into view
    driver.execute_script("arguments[0].scrollIntoView();", button)

    # Check for any potential overlapping elements and wait for them to disappear
    try:
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.XPATH, "//span[@tabindex='-1']"))
        )
    except TimeoutException:
        print("Potential overlapping element was not found or did not disappear.")
        print(driver.page_source)  # Print the current HTML content

    # Wait until the button is clickable
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Show all items in the list']")))
    except TimeoutException:
        print("Button is not clickable.")
        print(driver.page_source)  # Print the current HTML content
        return None

    # Click the button to reveal the hidden content
    try:
        button.click()
    except:
        print("Failed to click the button directly. Trying JavaScript click...")
        print(driver.page_source)  # Print the current HTML content
        # If direct click fails, use JavaScript to click
        driver.execute_script("arguments[0].click();", button)

    # Print the current URL to diagnose where the script has navigated to
    print("Current URL after clicking the button:", driver.current_url)

    # Wait for a short duration to ensure the content is revealed
    time.sleep(2)  # Adjust this duration if necessary

    # Now, scrape the characters' names
    try:
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='TruncatedContent__text TruncatedContent__text--small']/a"))
        )
        return [element.text.strip() for element in elements]
    except TimeoutException:
        print("Failed to locate the characters' section.")
        print(driver.page_source)  # Print the current HTML content
        return None



def extract_abstract(driver):
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".BookPageMetadataSection__description span.Formatted"))
        )
        return element.text.strip()
    except TimeoutException:
        return None

def extract_dispersion_of_scores(driver):
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='RatingStatistics__column']"))
        )
        return element.get_attribute('aria-label').strip()
    except TimeoutException:
        return None

success_count = 0
fail_count = 0

# Loop through each book

for book in book_details:
  try:
        driver.get(book['URL'])

        abstract = extract_abstract(driver)
        if abstract:
            print(f"Found abstract for book at {book['URL']}: {abstract}")
            success_count += 1
        else:
            print(f"Failed to find abstract info for book at {book['URL']}")
            fail_count += 1

        characters = extract_characters(driver)
        if characters:
            print(f"Found characters for book at {book['URL']}: {characters}")
            success_count += 1
        else:
            print(f"Failed to find characters info for book at {book['URL']}")
            fail_count += 1

        dispersion_of_scores = extract_dispersion_of_scores(driver)
        if dispersion_of_scores:
          print(f"Found dispersion of scores for book at {book['URL']}: {dispersion_of_scores}")
          success_count += 1
        else:
          print(f"Failed to find dispersion of scores info for book at {book['URL']}")
          fail_count += 1

        # Delay for 5 seconds
        time.sleep(5)

  except WebDriverException as e:
        print(f"An error occurred: {e}")
        fail_count += 1


print(f"Scraping complete. {success_count} successful scrapes, {fail_count} failed scrapes.")